<a href="https://colab.research.google.com/github/ishreiya27/curious/blob/master/Ass2_BigDataAnalytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
#Importing the data as a pandas dataframe and printing its shape

import pandas as pd
amazon_df = pd.read_csv("/content/Amazon_Responded_Oct05.csv", error_bad_lines=False) 
amazon_df.head()
amazon_df.shape

In [5]:
# Removing all Null values from the dataframe amazon_df and printing the new dimensions

amazon_clean = amazon_df.dropna(how="all")
amazon_clean.shape

(378132, 25)

In [0]:
DateField = amazon_clean["tweet_created_at"].str.split(" ", n = 6, expand = True) 
  
amazon_clean["WeekDay"]= DateField[0] 
amazon_clean["Month"]= DateField[1] 
amazon_clean["Date"]= DateField[2] 
amazon_clean["TimeStamp"]= DateField[3] 
amazon_clean["Geo"]= DateField[4] 
amazon_clean["Year"]= DateField[5] 



In [0]:
amazon_clean['Create_Date'] = amazon_clean[['Month', 'Date']].apply(lambda x: ' '.join(x), axis=1)

In [0]:
amazon_clean.head(5)

In [9]:
# Converting Pandas Dataframe into Spark RDD Dataframe
amazon_clean = amazon_clean.astype(str) # Converting pandas df to string first
Tweet_sdf = spark.createDataFrame(amazon_clean)
Tweet_sdf.show(10, False) # False allows us to show entire content of the columns

+--------------------+------------------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+----------------------------------------------------------------------------------------------------------------------------------------------+-------------+-------------+--------------------+------------------+------------------------------+--------------+------------------------------------------------------------------------------------------------------------------------------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+-------+-----+----+---------+-----+----+-----------+
|id_str              |tweet_created_at              |user_screen_name|user_id_str|user_statuses_count|user_favourites_

In [0]:
#file_loc = '/content/Amazon_Responded_Oct05.csv'

In [0]:
#TweetData = spark.read.csv(file_loc, inferSchema=True, header=True )

In [0]:
# Checking the datatype of our Spark RDD file Tweet_sdf
print(type(Tweet_sdf))

<class 'pyspark.sql.dataframe.DataFrame'>


In [0]:
# Checking the datatypes of each variable in Tweet_sdf
Tweet_sdf.printSchema()

In [0]:
# Printing the first 20 rows of Spark RDD Tweet_sdf
Tweet_sdf.show()

In [10]:
# Extracting columns 'id_str', 'tweet_created_at', 'user_verified', user_favourites_count', 'retweet_count' and 'text_' for our analysis
TweetDataCol = Tweet_sdf.select(Tweet_sdf.id_str, Tweet_sdf.Create_Date, Tweet_sdf.user_verified, Tweet_sdf.user_favourites_count, Tweet_sdf.retweet_count, Tweet_sdf.text_)
TweetDataCol.show(20, False)

+--------------------+-----------+-------------+---------------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|id_str              |Create_Date|user_verified|user_favourites_count|retweet_count|text_                                                                                                                                                        |
+--------------------+-----------+-------------+---------------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|'793270689780203520'|Nov 01     |False        |4079                 |0            |@AmazonHelp Can you please DM me? A product I ordered last year never arrived.                                                                               |
|'793281386912354304'|Nov 01

In [0]:
# Step 1: Remove the records where “user_verified” is “FALSE”.
# Using Filter function to remove the rows with “user_verified” as “FALSE”

TweetDatafilter = TweetDataCol.rdd.filter(lambda TweetDataCol:'True' in  TweetDataCol[2])

In [0]:
# Printing 5 rows of TweetDatafilter to check the filter
TweetDatafilter.take(5)

In [0]:
type(TweetDatafilter)

pyspark.rdd.PipelinedRDD

In [13]:
#print(type(TweetDatafilter))
TweetDatafilter.count()

171898

In [14]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

#sc = SparkContext()
#tweets = spark.sparkContext.parallelize(TweetDatafilter.collect())

TweetDatafilter = TweetDatafilter.toDF()
type(TweetDatafilter)

pyspark.sql.dataframe.DataFrame

In [0]:
# Task 2: For the remaining records (“user_verified” is “TRUE”), group by created date, and count the number of tweets for each date.

import pyspark.sql.functions as f

groupeByDate = TweetDatafilter.groupby('Create_Date').count().withColumnRenamed("count","tweet_count")

In [16]:
groupeByDate.sort((groupeByDate.tweet_count).desc()).show(100)

+-----------+-----------+
|Create_Date|tweet_count|
+-----------+-----------+
|     Jan 03|       1536|
|     Jan 10|       1508|
|     Jan 11|       1496|
|     Jan 12|       1410|
|     Jan 06|       1364|
|     Jan 07|       1360|
|     Jan 20|       1342|
|     Mar 02|       1298|
|     Jan 13|       1295|
|     Jan 21|       1292|
|     Jan 14|       1290|
|     Jan 18|       1286|
|     Dec 15|       1279|
|     Jan 24|       1259|
|     Nov 18|       1249|
|     Dec 03|       1201|
|     Jan 02|       1196|
|     Jun 27|       1192|
|     Jul 04|       1190|
|     Jan 19|       1175|
|     Jan 25|       1163|
|     Jan 23|       1149|
|     Jan 08|       1143|
|     Jan 17|       1124|
|     May 11|       1120|
|     Jul 03|       1112|
|     Mar 30|       1109|
|     Apr 05|       1089|
|     Jan 26|       1083|
|     Jan 27|       1080|
|     Jun 20|       1074|
|     Jan 04|       1071|
|     Nov 05|       1058|
|     Feb 22|       1058|
|     Feb 09|       1054|
|     Jun 23

In [0]:
# Task 3

In [0]:
# Filter the records with the date of highest number of tweets, i.e., Jan 03

Task3Data = TweetDatafilter

Task3Data = Task3Data.rdd.filter(lambda Task3Data:'Jan 03' in  Task3Data[1])


In [18]:
Task3Data.take(5)

[Row(id_str="'816139627270721537'", Create_Date='Jan 03', user_verified='True', user_favourites_count='11066', retweet_count='0', text_="@mcp111 Hey! We'll have this looked into internally. Thanks for your patience. ^HK"),
 Row(id_str="'816357712732819467'", Create_Date='Jan 03', user_verified='True', user_favourites_count='11066', retweet_count='0', text_="@mcp111 Hi there, you can change the country from the settings. Please reach us via the link provided earlier and we'll help you out, ^VM"),
 Row(id_str="'816167583263162369'", Create_Date='Jan 03', user_verified='True', user_favourites_count='11081', retweet_count='0', text_="@stevebrowne Hi Steve, sorry to hear this. As we can't view accounts here, please contact here for more options: https://t.co/JzP7hlA23B ^CJ"),
 Row(id_str="'816326113660174336'", Create_Date='Jan 03', user_verified='True', user_favourites_count='11066', retweet_count='0', text_="@boozysmurf We'd like the chance to look into this w/ you &amp; investigate furth

In [19]:
Task3Data = Task3Data.toDF()
type(Task3Data)

pyspark.sql.dataframe.DataFrame

In [0]:

from pyspark.sql.types import IntegerType
Task3Data = Task3Data.withColumn("user_favourites_count", Task3Data["user_favourites_count"].cast(IntegerType()))
Task3Data = Task3Data.withColumn("retweet_count", Task3Data["retweet_count"].cast(IntegerType()))


In [0]:
Task3Data.printSchema()

root
 |-- id_str: string (nullable = true)
 |-- Create_Date: string (nullable = true)
 |-- user_verified: string (nullable = true)
 |-- user_favourites_count: integer (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- text_: string (nullable = true)



In [0]:
Task3Data = Task3Data.withColumn('Sum_favourites_retweets', Task3Data.user_favourites_count+ Task3Data.retweet_count)

In [22]:
Task3Data.take(5)

[Row(id_str="'816139627270721537'", Create_Date='Jan 03', user_verified='True', user_favourites_count=11066, retweet_count=0, text_="@mcp111 Hey! We'll have this looked into internally. Thanks for your patience. ^HK", Sum_favourites_retweets=11066),
 Row(id_str="'816357712732819467'", Create_Date='Jan 03', user_verified='True', user_favourites_count=11066, retweet_count=0, text_="@mcp111 Hi there, you can change the country from the settings. Please reach us via the link provided earlier and we'll help you out, ^VM", Sum_favourites_retweets=11066),
 Row(id_str="'816167583263162369'", Create_Date='Jan 03', user_verified='True', user_favourites_count=11081, retweet_count=0, text_="@stevebrowne Hi Steve, sorry to hear this. As we can't view accounts here, please contact here for more options: https://t.co/JzP7hlA23B ^CJ", Sum_favourites_retweets=11081),
 Row(id_str="'816326113660174336'", Create_Date='Jan 03', user_verified='True', user_favourites_count=11066, retweet_count=0, text_="@boo

In [0]:
from pyspark.sql.functions import desc 
Task3DataTop100 = Task3Data.sort(desc("Sum_favourites_retweets")).limit(100)

In [24]:
Task3DataTop100.show()

+--------------------+-----------+-------------+---------------------+-------------+--------------------+-----------------------+
|              id_str|Create_Date|user_verified|user_favourites_count|retweet_count|               text_|Sum_favourites_retweets|
+--------------------+-----------+-------------+---------------------+-------------+--------------------+-----------------------+
|'816356191035068422'|     Jan 03|         True|                13678|            0|@AmazonHelp This ...|                  13678|
|'816356580950163457'|     Jan 03|         True|                13678|            0|@AmazonHelp Pleas...|                  13678|
|'816296887921836033'|     Jan 03|         True|                11196|            0|@AmazonHelp hi th...|                  11196|
|'816083406962434048'|     Jan 03|         True|                11081|            1|@ItsJosshA We alw...|                  11082|
|'816095108013654017'|     Jan 03|         True|                11081|            1|@KStef

In [25]:
Task3Data_tweets = Task3DataTop100.select("text_").rdd.flatMap(lambda x: x).collect()
Task3Data_tweets[0:5]

['@AmazonHelp This is becoming a joke now. I need to speak to a supervisor. Your colleagues keep trying to put me through then cutting me off',
 '@AmazonHelp Please don’t suggest I call you again. You need to arrange for a supervisor to call me asap. DM please',
 "@AmazonHelp hi there, seeing a monthly charge on my CC from Amazon but can't find any subscriptions in my account. help?",
 '@ItsJosshA We always aim to deliver by the date given in your confirmation email. Have we missed that date? Any update in tracking?  ^NF',
 '@KStefl Sounds like you know what to add to your Halloween playlist for this year! ^BV']

In [0]:
# Creating Spark Context
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [27]:
# Reading 'tweet' to a RDD object using spark context
tweet_rdd = sc.parallelize(Task3Data_tweets)
tweet_rdd.take(5)

['@AmazonHelp This is becoming a joke now. I need to speak to a supervisor. Your colleagues keep trying to put me through then cutting me off',
 '@AmazonHelp Please don’t suggest I call you again. You need to arrange for a supervisor to call me asap. DM please',
 "@AmazonHelp hi there, seeing a monthly charge on my CC from Amazon but can't find any subscriptions in my account. help?",
 '@ItsJosshA We always aim to deliver by the date given in your confirmation email. Have we missed that date? Any update in tracking?  ^NF',
 '@KStefl Sounds like you know what to add to your Halloween playlist for this year! ^BV']

In [0]:
import string
import re

def clean_tweet(x):
  
  # Delete all the URLs in the tweets
  text00 = re.sub(r'www\S+', '', x)
  text01 = re.sub(r'http\S+', '', text00)
  
  # Delete all the numbers in the tweets
  text1 = ''.join([i for i in text01 if not i.isdigit()])
  
  # Delete all the punctuation marks in the tweets
  text2 = text1.translate(str.maketrans('','',string.punctuation))
  
  # Convert text to LOWERCASE
  text3 = text2.lower()

  return text3

In [29]:
# Cleaning the text of tweets: 1. Removing URLs, 2. Removing non-alphabets, 3. Lowercase 
clean_tweet_rdd = tweet_rdd.map(clean_tweet)
clean_tweet_rdd.take(10)

['amazonhelp this is becoming a joke now i need to speak to a supervisor your colleagues keep trying to put me through then cutting me off',
 'amazonhelp please don’t suggest i call you again you need to arrange for a supervisor to call me asap dm please',
 'amazonhelp hi there seeing a monthly charge on my cc from amazon but cant find any subscriptions in my account help',
 'itsjossha we always aim to deliver by the date given in your confirmation email have we missed that date any update in tracking  nf',
 'kstefl sounds like you know what to add to your halloween playlist for this year bv',
 'schoey happy birthday matt friendsforever friendshipgoals jo',
 'ratbones you so fancy you already knooow ep',
 'zesaiyan well certainly try to get it to you danny has the tracking updated as out for delivery here  hb',
 'kbsarfaraz also please dont provide the order details as we consider it personal information ht ',
 'kckvn hi sorry about that being a public platform well not be able to acce

In [30]:
# Building Map function
map = clean_tweet_rdd.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1))
map.take(5)

[('amazonhelp', 1), ('this', 1), ('is', 1), ('becoming', 1), ('a', 1)]

In [31]:
# Building Reduce function
counts = map.reduceByKey(lambda a, b: a + b)
counts.take(5)

[('this', 25), ('is', 15), ('becoming', 1), ('joke', 1), ('now', 2)]

In [32]:
# Total number of distinct words
print(len(counts.collect()))

497


In [33]:
# Getting the Top 10 most popular words and their words frequency
# Sorting 'counts' in descending order and getting the first 10 elements
countsSorted = counts.takeOrdered(497) # Conditioned on that number of characters in the string should be at least 1
countsSorted

[('', 79),
 ('a', 20),
 ('ab', 1),
 ('able', 4),
 ('about', 4),
 ('above', 1),
 ('access', 2),
 ('account', 6),
 ('accounts', 1),
 ('add', 1),
 ('additional', 1),
 ('address', 1),
 ('af', 7),
 ('after', 3),
 ('again', 3),
 ('aim', 3),
 ('ajfus', 1),
 ('ak', 3),
 ('already', 1),
 ('also', 3),
 ('always', 2),
 ('am', 2),
 ('amazon', 2),
 ('amazonhelp', 3),
 ('amp', 2),
 ('an', 3),
 ('and', 10),
 ('anki', 1),
 ('another', 1),
 ('any', 12),
 ('anything', 2),
 ('aoamlibrarian', 1),
 ('apologies', 3),
 ('app', 1),
 ('appreciate', 1),
 ('ar', 10),
 ('are', 3),
 ('armchairdj', 1),
 ('arrange', 1),
 ('arrive', 7),
 ('arrived', 1),
 ('as', 4),
 ('asap', 1),
 ('asinalexander', 2),
 ('assist', 2),
 ('assistance', 2),
 ('at', 3),
 ('available', 1),
 ('bank', 1),
 ('be', 10),
 ('becoming', 1),
 ('been', 1),
 ('before', 1),
 ('being', 1),
 ('below', 1),
 ('better', 1),
 ('birthday', 1),
 ('bourge', 2),
 ('briancranfill', 2),
 ('browsers', 1),
 ('business', 5),
 ('but', 2),
 ('bv', 4),
 ('by', 7),
 ('

In [34]:
## Converting the final Map-reduce output into a data frame
import pandas as pd

# Creating a dataframe object from listoftuples
MapReduceOutput = pd.DataFrame(countsSorted, columns = ['Tweet_Word' , "Frequency"])

print(MapReduceOutput)

MapReduceOutput.head()


    Tweet_Word  Frequency
0                      79
1            a         20
2           ab          1
3         able          4
4        about          4
..         ...        ...
492      youve          1
493         yp          1
494   zesaiyan          1
495    zlgreen          4
496        👍bv          1

[497 rows x 2 columns]


,Tweet_Word,Frequency
0,,79
1,a,20
2,ab,1
3,able,4
4,about,4


In [0]:
MapReduceOutput.to_excel(r"C:\Users\ishre\Desktop\IDS 561\Assignments\Assignment 2\Ass2_Mapreduce.xlsx")

In [0]:
#Part 2

In [0]:
#Renaming id_str column to tweet_id in Tweet_sdf to remove ambiguity

Tweet_sdf = Tweet_sdf.withColumnRenamed("id_str", "tweet_id")

In [41]:
#Importing the data as a pandas dataframe and printing its shape

import pandas as pd
find_text = pd.read_csv("/content/find_text.csv") 
find_text.head()
find_text.shape

(53928, 2)

In [39]:
# Converting Pandas Dataframe into Spark RDD Dataframe

find_text = find_text.astype(str) # Converting pandas df to string first
find_text_sdf = spark.createDataFrame(find_text)
find_text_sdf.show(10, False) # False allows us to show entire content of the columns

+--------------------+----+
|id_str              |text|
+--------------------+----+
|'793270689780203520'|nan |
|'793281386912354304'|nan |
|'793299404975247360'|nan |
|'793301295255945216'|nan |
|'793315815411978240'|nan |
|'793322306848292864'|nan |
|'793322433625415680'|nan |
|'793365409047023616'|nan |
|'793369654878232577'|nan |
|'793375905280393216'|nan |
+--------------------+----+
only showing top 10 rows



In [42]:
#Joining find_text_sdf with Tweet_sdf according to id_str

left_join = find_text_sdf.join(Tweet_sdf, find_text_sdf.id_str == Tweet_sdf.tweet_id, how='left') # Could also use 'left_outer'
left_join.show()

+--------------------+----+--------------------+--------------------+----------------+-----------+-------------------+---------------------+--------------+-----------------+--------------+--------------------+--------------------+-------------+--------------------+------------------+--------------------+--------------+--------------------+--------------+---------+-----------------------+-------------------------+-----------------------+-------------+---------+--------------------+-------+-----+----+---------+-----+----+-----------+
|              id_str|text|            tweet_id|    tweet_created_at|user_screen_name|user_id_str|user_statuses_count|user_favourites_count|user_protected|user_listed_count|user_following|    user_description|       user_location|user_verified|user_followers_count|user_friends_count|     user_created_at|tweet_language|               text_|favorite_count|favorited|in_reply_to_screen_name|in_reply_to_status_id_str|in_reply_to_user_id_str|retweet_count|retweete

In [43]:
#Selecting id_str and text_ from the joined tables

left_join_out = left_join.select(left_join.id_str, left_join.text_)

#TweetDataCol = Tweet_sdf.select(Tweet_sdf.id_str, Tweet_sdf.Create_Date, Tweet_sdf.user_verified, Tweet_sdf.user_favourites_count, Tweet_sdf.retweet_count, Tweet_sdf.text_)
left_join_out.show(20, False)

+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|id_str              |text_                                                                                                                                                     |
+--------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------+
|'793552243689676800'|@AmazonHelp amazon have made and admitted to a series of error that have impacted a myself and a colleague                                                |
|'793595479116439552'|@SuNoSuKo Please don't provide your order details we consider them as personal info. Our twitter page is visible to public. ^VM                           |
|'793751757247619072'|@aasifkhan Hey, I'm sorry for this. Product should reach the fulfillment center, when it

In [44]:
join_output = left_join_out.select("*").toPandas()

print(join_output)

                     id_str                                              text_
0      '793552243689676800'  @AmazonHelp amazon have made and admitted to a...
1      '793595479116439552'  @SuNoSuKo Please don't provide your order deta...
2      '793751757247619072'  @aasifkhan Hey, I'm sorry for this. Product sh...
3      '793788930361556996'  @AmazonHelp I would get more assistance from t...
4      '793797330310983680'  Disgusting service @amazonIN\r\nProducts are n...
...                     ...                                                ...
53923  '809401768211685376'  @AmazonVideoIN Censored content, no support fo...
53924  '809412394212466689'  @AmazonHelp @rathipoonam please share details,...
53925  '809449404801421314'  @SlapdashGourmet We'd like to help. Are all it...
53926  '809513226144804864'  @deecaltee When did you reach out to the selle...
53927  '809729921320882179'  @engeetham Hi, sorry for the long stretch. We'...

[53928 rows x 2 columns]


In [0]:
#Exporting output to excel
join_output.to_excel(r"C:\Users\ishre\Desktop\IDS 561\Assignments\Assignment 2\find_text_output.xlsx")